In [ ]:
from joblib import dump, load

In [ ]:
# !pip install sklearn_crfsuite
# !pip install regex
# !pip install -U 'scikit-learn<0.24'

In [1]:
import re
import sklearn
import sklearn_crfsuite
import string
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn_crfsuite import metrics

In [2]:
def read(file):
    with open(file, 'r', encoding='utf8') as f:
        file = f.read().splitlines()
    data = [[] for _ in range(len(file))]
    for idx, i in enumerate(file):
        a = i.split()
        for j in a:
            tmp = (j.rsplit('/',1))
            data[idx].append((tmp[0], tmp[1]))
    return data

In [3]:
train_set = read('C:/Users/owcap/Documents/Learning/CS221/Project/corpus/train.txt')
test_set = read('C:/Users/owcap/Documents/Learning/CS221/Project/corpus/test.txt')

In [4]:
def word2features(sent, i):
    word = sent[i][0]
#     postag = sent[i][1]
    if i == 0:
        first = True
    else:
        first = False
    if i == len(sent) - 1:
        last = True
    else:
        last = False

    features = {
        'word' : word,
        'word.lower()': word.lower(),
        'number': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word.isupper()': word.isupper(),
        'has_hyphen': '-' in word,
        'is_first': first,
        'is_last': last
    }

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [word[1] for word in sent]


def sent2tokens(sent):
    return [word[0] for word in sent]

In [5]:
X_train = [sent2features(s) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s) for s in test_set]
y_test = [sent2labels(s) for s in test_set]

In [6]:
def word2featuresss(sent, i):
    word = sent[i]
#     postag = sent[i][1]
    if i == 0:
        first = True
    else:
        first = False
    if i == len(sent) - 1:
        last = True
    else:
        last = False

    features = {
        'word' : word,
        'word.lower()': word.lower(),
        'number': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word.isupper()': word.isupper(),
        'has_hyphen': '-' in word,
        'is_first': first,
        'is_last': last
    }

    return features

In [7]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1,
    c2=0.25,
)

In [8]:
crf.fit(X_train, y_train)

c:\users\owcap\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', c1=0.1, c2=0.25, keep_tempfiles=None)

In [ ]:
dump(crf,'crf.joblib')

In [9]:
k = 'The cat is sitting on the couch .'.split()

In [12]:
a = [word2featuresss(k, idx) for idx, i in enumerate(k)]

In [15]:
y__ = crf.predict([a])

In [16]:
y__

[['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT']]

In [ ]:
#obtaining metrics such as accuracy, etc. on the train set
labels = list(crf.classes_)
labels.remove('X')

ypred = crf.predict(X_train)
print('F1 score on the train set = {}\n'.format(metrics.flat_f1_score(y_train, ypred, average='weighted', labels=labels)))
print('Accuracy on the train set = {}\n'.format(metrics.flat_accuracy_score(y_train, ypred)))

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [ ]:
print('Train set classification report: \n\n{}'.format(metrics.flat_classification_report(
y_train, ypred, labels=sorted_labels, digits=3
)))
#obtaining metrics such as accuracy, etc. on the test set
ypred = crf.predict(X_test)
print('F1 score on the test set = {}\n'.format(metrics.flat_f1_score(y_test, ypred,
average='weighted', labels=labels)))
print('Accuracy on the test set = {}\n'.format(metrics.flat_accuracy_score(y_test, ypred)))

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
# print('Test set classification report: \n\n{}'.format(metrics.flat_classification_report(y_test, ypred, labels=sorted_labels, digits=3)))